# Visualization Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [22]:
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from src.pipeline.flows.main import main_flow

In [4]:
_ = main_flow(max_plays=3)

19:05:42.973 | INFO    | prefect.engine - Created flow run 'sticky-cougar' for flow 'main-flow'

19:05:43.189 | INFO    | Flow run 'sticky-cougar' - Created task run 'read_csv-3609c996-0' for task 'read_csv'

19:05:43.192 | INFO    | Flow run 'sticky-cougar' - Executing 'read_csv-3609c996-0' immediately...

19:05:43.458 | INFO    | Task run 'read_csv-3609c996-0' - Finished in state Completed()

19:05:43.490 | INFO    | Flow run 'sticky-cougar' - Created task run 'read_csv-3609c996-1' for task 'read_csv'

...

19:05:46.002 | INFO    | Task run 'write_csv-386fe2af-0' - Finished in state Completed()

19:05:46.031 | INFO    | Flow run 'sticky-cougar' - Created task run 'write_csv-386fe2af-1' for task 'write_csv'

19:05:46.033 | INFO    | Flow run 'sticky-cougar' - Executing 'write_csv-386fe2af-1' immediately...

19:05:46.104 | INFO    | Task run 'write_csv-386fe2af-1' - Finished in state Completed()

19:05:46.140 | INFO    | Flow run 'sticky-cougar' - Finished in state Completed('All states completed.')

In [15]:
df_frame_records_all = pd.read_csv(f"{DIR}/data/outputs/frame_records.csv")
df_frame_records_all["records"] = df_frame_records_all["records"].apply(literal_eval)

df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

In [19]:
game_id = 2021090900
play_id = 97
query = f"gameId == {game_id} and playId == {play_id}"

df_frame_records = df_frame_records_all.query(query)
df_frame_area = df_areas_all.query(query)

In [21]:
df_frame_area.head()

,gameId,playId,frameId,method,pocket,area
0,2021090900,97,1,passer_radius,"{'area': 104.72938933713064, 'metadata': {'ver...",104.729389
1,2021090900,97,2,passer_radius,"{'area': 104.01436284917375, 'metadata': {'ver...",104.014363
2,2021090900,97,3,passer_radius,"{'area': 103.95687170361305, 'metadata': {'ver...",103.956872
3,2021090900,97,4,passer_radius,"{'area': 104.9172565778155, 'metadata': {'vert...",104.917257
4,2021090900,97,5,passer_radius,"{'area': 105.33257512661999, 'metadata': {'ver...",105.332575


In [ ]:
def gen_frames(input_rows):
    rows = []
    curr_frame = 1
    for row in input_rows:
        frame = row["frameId"]
        if curr_frame == frame:
            rows.append(row)
        else:
            # Stop adding to this frame and yield it
            yield rows
            # Reset for the next frame
            curr_frame = frame
            rows = [row]
    # Yield the last frame
    yield rows
    # Yield terminal value
    yield None

    
def animate_play(df_play_tracking_normalized):
    """Sorry for the unreadable code, I'll clean it up later."""
    team_options = list({team for team in df_play_tracking_normalized["team"].unique() if team != "football"})
    team_a = df_play_tracking_normalized["team"] == team_options[0]
    team_b = df_play_tracking_normalized["team"] == team_options[1]

    x = df_play_tracking_normalized["nx"]
    y = df_play_tracking_normalized["ny"]
    g_min = min(x.min(), y.min())
    g_max = max(x.max(), y.max())

    ratio = 0.4
    dim = ratio * (g_max - g_min)
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(dim, dim)

    ax.axvline(0, color="gray", dashes=[5, 1])
    ax.axhline(0, color="gray", dashes=[5, 1])

    trails = pd.DataFrame(df_play_tracking_normalized.query("team != 'football' and pff_role in ('Pass', 'Pass Block', 'Pass Rush')"))
    trails["color"] = trails["team"].apply(lambda team: "red" if team == team_options[0] else "blue")

    ax.scatter(trails["nx"], trails["ny"], s=1, color=trails["color"])

    ax.set_xlim(g_min - 1, g_max + 1)
    ax.set_ylim(g_min - 1, g_max + 1)

    df_sorted = df_play_tracking_normalized.sort_values(by="frameId", ascending=True)
    rows = []
    for r in df_sorted.to_dict(orient="records"):
        team = r["team"]
        if team == "football":
            row = {
                "frameId": r["frameId"],
                "pid": "football",
                "x": r["nx"],
                "y": r["ny"],
            }
            rows.append(row)
            continue
        jersey = str(int(r["jerseyNumber"]))
        row = {
            "frameId": r["frameId"],
            "pid": f"{team} {jersey}",
            "j": jersey,
            "x": r["nx"],
            "y": r["ny"],
            "o": r["no"],
            "dir": r["ndir"],
            "c": "red" if team == team_options[0] else "blue",
        }
        rows.append(row)

    frame_list = list(gen_frames(rows))

    diameter = 1.125
    font_size = 12 * diameter
    radius = 0.5 * diameter
    y_offset = 0.175 * diameter

    ball = None
    jerseys = {}
    numbers = {}
    orientations = {}
    directions = {}
    for p in frame_list[0]:
        pid = p["pid"]
        x, y = p["x"], p["y"]

        if pid == "football":
            ball = plt.Circle((x, y), radius * 0.5, color="brown")
            continue

        x_offset = (0.375 if len(p["j"]) > 1 else 0.2) * diameter

        degrees = p["o"]
        radians = math.radians((-1 * degrees) + 90)
        dx = math.cos(radians)
        dy = math.sin(radians)
        orientation = ax.arrow(x, y, dx, dy, color=p["c"])
        orientations[pid] = orientation

        degrees = p["dir"]
        radians = math.radians((-1 * degrees) + 90)
        dx = math.cos(radians)
        dy = math.sin(radians)
        direction = ax.arrow(x, y, dx, dy, color="lightgray")
        directions[pid] = direction

        circle = plt.Circle((x, y), radius, color=p["c"])
        ax.add_patch(circle)
        jerseys[pid] = circle

        text = ax.text(x - x_offset, y - y_offset, p["j"], fontsize=font_size, color="white")
        numbers[pid] = text

    ax.add_patch(ball)

    
    def animate(i):
        if i >= len(frame_list):
            return ()
        frame_rows = frame_list[i] or []
        for row in frame_rows:
            pid = row["pid"]
            x, y = row["x"], row["y"]\

            if pid == "football":
                ball.center = x, y
                continue

            j = jerseys[pid]    
            j.center = x, y

            n = numbers[pid]
            x_offset = (0.375 if len(row["j"]) > 1 else 0.2) * diameter
            n.set_x(x - x_offset)
            n.set_y(y - y_offset)

            o = orientations[pid]
            degrees = row["o"]
            radians = math.radians((-1 * degrees) + 90)
            dx = math.cos(radians)
            dy = math.sin(radians)
            o.set_data(x=x, y=y, dx=dx, dy=dy)

            d = directions[pid]
            degrees = row["dir"]
            radians = math.radians((-1 * degrees) + 90)
            dx = math.cos(radians)
            dy = math.sin(radians)
            d.set_data(x=x, y=y, dx=dx, dy=dy)

        return (*jerseys.values(), *numbers.values(), *orientations.values(), *directions.values())


    ani = animation.FuncAnimation(fig, animate, frames=range(len(frame_list) + 1), interval=100, blit=True)
    plt.close("all")
    video = ani.to_html5_video()
    return HTML(video)